In [70]:
import os
from cloudvolume import CloudVolume
from skimage import measure, io

prep_id = 'DK59'
channel = 1
INPUT = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{prep_id}/preps/CH1/thumbnail_aligned'
OUTPUT_DIR = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{prep_id}/neuroglancer_data/test'
FINAL_OUTPUT_DIR = os.path.join(OUTPUT_DIR, f"C{channel}T")


def sizeof_fmt(num, suffix="B"):
    for unit in ["", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"]:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"

    
os.makedirs(FINAL_OUTPUT_DIR, exist_ok=True)
infiles = os.listdir(INPUT)

print(f"INPUT FOLDER: {INPUT}")
print(f"CURRENT FILE COUNT: {len(infiles)}")
print(f"OUTPUT FOLDER: {OUTPUT_DIR}")

resolution_scales = [325, 325, 20000]
chunk_size = [512, 512, 16] # units are voxels [recommended powers of 2; 2**9=512]



#for infile in infiles:
infile = os.path.join(INPUT, '000.tif') 
index = int(os.path.splitext(os.path.basename(infile))[0])

try:
    img = io.imread(infile, img_num=0)
    print(f"FILE READ: {infile}")
    # print(f"RAW SIZE: {sizeof_fmt(rbytes)}")
    # sparse_img = sparse.COO(img)
    # sparse_bytes = sparse_img.nbytes
    # print(f"SPARSE SIZE: {sizeof_fmt(sparse_bytes)}")
except IOError as ioe:
    print(f'could not open {infile} {ioe}')
try:
    img = img.reshape(channel, img.shape[0], img.shape[1]).T
    volume_and_image_size = img.shape
    #volume_and_image_size = (img.shape[0], img.shape[1], len(infiles))
    print(f"VOLUME DIMENSIONS: {volume_and_image_size}")
except:
    print(f'could not reshape {infile}')

info = CloudVolume.create_new_info(
    num_channels    = 1,
    layer_type      = 'image',
    data_type       = 'uint16', # Channel images might be 'uint8'
    # raw, png, jpeg, compressed_segmentation, fpzip, kempressed, compresso
    encoding        = 'raw', 
    resolution      = resolution_scales, # Voxel scaling, units are in nanometers
    voxel_offset    = [0, 0, 0], # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size      = chunk_size, # units are voxels
    volume_size     = volume_and_image_size, # e.g. a cubic millimeter dataset
)
precomputed_vol = CloudVolume(f'file://{FINAL_OUTPUT_DIR}', mip=0, info=info, delete_black_uploads=True, compress=True, progress=True, parallel=True)
precomputed_vol.commit_info()
precomputed_vol.parallel = 15




try:
    print(f"TRYING TO ADD {type(img), img.shape} TO INDEX: {index}")
    precomputed_vol[:, :, index] = img
        # if orientation == 'sagittal':
        #     self.precomputed_vol[:, :, index] = img
        # elif orientation == 'coronal':
        #     self.precomputed_vol[index, :, :] = img 
        # elif orientation == 'horizontal':
        #     self.precomputed_vol[:, index, :] = img
except:
    print(f'could not set {infile} to precomputed')

precomputed_vol.commit_provenance()
precomputed_vol.cache.flush()
    

INPUT FOLDER: /net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK59/preps/CH1/full_aligned
CURRENT FILE COUNT: 388
OUTPUT FOLDER: /net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK59/neuroglancer_data/test
FILE READ: /net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK59/preps/CH1/full_aligned/000.tif
VOLUME DIMENSIONS: (68157, 31465, 1)
TRYING TO ADD (<class 'numpy.ndarray'>, (68157, 31465, 1)) TO INDEX: 0


Upload: 100%|█████████▉| 8293/8308 [00:44<00:00, 229.69it/s]

In [66]:
del img
print(img.shape)
#print(sparse_img.shape)
#sparse_img.reshape(channel, sparse_img.shape[0], sparse_img.shape[1]).T
#print(sparse_img.shape)
#volume_and_image_size
#precomputed_vol[:, :, index] = img

#sparse_img.reshape(channel, img.shape[0], img.shape[1]).T
#print(index,type(index))
#infile = os.path.join(INPUT, '000.tif') #virtually empty file
#index = os.path.splitext(infile)[0]

#precomputed_vol[:, :, index] = img

NameError: name 'img' is not defined